# 简介

## 设置

In [2]:
from __future__ import  absolute_import
from __future__ import  division
from __future__ import print_function

import tensorflow as tf
import numpy as np

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## 张量值

TensorFlow 中的核心数据单位是张量。一个张量由一组形成阵列（任意维数）的原始值组成。张量的阶是它的维数，而它的形状是一个整数元组，指定了阵列每个维度的长度。

In [3]:
3. # a rank 0 tensor; a scalar with shape [],
[1., 2., 3.] # a rank 1 tensor; a vector with shape [3]
[[1., 2., 3.], [4., 5., 6.]] # a rank 2 tensor; a matrix with shape [2, 3]
[[[1., 2., 3.]], [[7., 8., 9.]]] # a rank 3 tensor with shape [2, 1, 3]

[[[1.0, 2.0, 3.0]], [[7.0, 8.0, 9.0]]]

## TensorFlow Core 演示

可以将 TensorFlow Core 程序看作由两个互相独立的部分组成：

1.构建计算图 (tf.Graph)。
2.运行计算图（使用 tf.Session）。

### 图

计算图是排列成一个图的一系列 TensorFlow 指令。图由两种类型的对象组成。

操作（简称“op”）：图的节点。操作描述了消耗和生成张量的计算。

张量：图的边。它们代表将流经图的值。大多数 TensorFlow 函数会返回 tf.Tensors。

In [6]:
a = tf.constant(3.0,dtype=tf.float32)
b = tf.constant(4.0)
total = a + b
print(a)
print(b)
print(total)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


图中的每个指令都拥有唯一的名称。这个名称不同于使用 Python 分配给相应对象的名称。张量是根据生成它们的指令命名的，后面跟着输出索引，如上文的 "add:0" 所示。

## TensorBoard

TensorFlow 提供了一个名为 TensorBoard 的实用程序。TensorBoard 的诸多功能之一是将计算图可视化。您只需要使用几个简单的命令就能轻松完成此操作。

In [7]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

## 会话（Session）

要评估张量，您需要实例化一个 tf.Session 对象（非正式名称为会话）。会话会封装 TensorFlow 运行时的状态，并运行 TensorFlow 操作。如果说 tf.Graph 像一个 .py 文件，那么 tf.Session 就像一个 python 可执行对象。

In [8]:
sess = tf.Session()
print(sess.run(total))

7.0


In [11]:
print(sess.run({'ab':(a,b),"total":total}))

{'ab': (3.0, 4.0), 'total': 7.0}


In [14]:
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
print(sess.run((out1,out2)))

[0.85014737 0.6020273  0.3850925 ]
[0.6006186  0.05520439 0.6114532 ]
(array([1.8606313, 1.4063618, 1.3833532], dtype=float32), array([2.8606315, 2.4063618, 2.3833532], dtype=float32))


## 供给

目前来讲，这个图不是特别有趣，因为它总是生成一个常量结果。图可以参数化以便接受外部输入，也称为占位符。占位符表示承诺在稍后提供值，它就像函数参数。

In [15]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

In [16]:
print(sess.run(z,feed_dict={x:3,y:4.5}))
print(sess.run(z,feed_dict={x:[1,3],y:[2,4]}))

7.5
[3. 7.]


## 数据集

占位符适用于简单的实验，但数据集是将数据流式传输到模型的首选方法。

要从数据集中获取可运行的 tf.Tensor，您必须先将其转换成 tf.data.Iterator，然后调用迭代器的 get_next 方法。

创建迭代器的最简单的方式是采用 make_one_shot_iterator 方法。例如，在下面的代码中，next_item 张量将在每次 run 调用时从 my_data 阵列返回一行：

In [19]:
my_data = [
    [0,1,],
    [2,3,],
    [4,5,],
    [6,7,],
]
slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

到达数据流末端时，Dataset 会抛出 OutOfRangeError。例如，下面的代码会一直读取 next_item，直到没有数据可读

In [20]:
while True:
    try:
        print(sess.run(next_item))
    except tf.errors.OutOfRangeError:
        break

[0 1]
[2 3]
[4 5]
[6 7]


In [28]:
r = tf.random_normal([10,3])
dataset = tf.data.Dataset.from_tensor_slices(r)
iterator = dataset.make_initializable_iterator()
next_row = iterator.get_next()

sess.run(iterator.initializer)
while True:
    try:
        print(sess.run(next_row))
    except tf.errors.OutOfRangeError:
        break

[0.2667524 1.2624987 1.2931088]
[-1.3710983 -1.759538  -2.0084453]
[ 1.0355133   0.22697236 -0.4193986 ]
[-2.276799   0.9500257 -1.0041165]
[-0.06327701  0.29767665  0.2769463 ]
[ 1.3498988  -1.0641057  -0.18013124]
[0.5873358 0.5025349 1.3213618]
[-0.76654744 -0.39083168  1.1014665 ]
[-0.23919052  0.4992199   0.67420566]
[-1.0455903  1.1864312  1.3076091]


## 层

层将变量和作用于它们的操作打包在一起。例如，密集连接层会对每个输出对应的所有输入执行加权和，并应用激活函数（可选）。连接权重和偏差由层对象管理。

## 创建层

In [25]:
x = tf.placeholder(tf.float32,shape=[None,3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)

## 初始化层

In [26]:
init = tf.global_variables_initializer()
sess.run(init)

## 执行层

In [29]:
print(sess.run(y,feed_dict={x:[[1,2,3],[4,5,6]]}))

[[2.2522702]
 [2.5093665]]


## 层函数的快捷方式

In [30]:
x = tf.placeholder(tf.float32,shape=[None,3])
y =  tf.layers.dense(x,units=1)

init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y,feed_dict={x:[[1,2,3],[4,5,6]]}))

[[1.0257243]
 [2.5385725]]


## 特征列

使用特征列进行实验的最简单方法是使用 tf.feature_column.input_layer 函数。此函数只接受密集列作为输入，因此要查看类别列的结果，您必须将其封装在 tf.feature_column.indicator_column 中。

In [32]:
features = {
    'sales' : [[5], [10], [8], [9]],
    'department': ['sports', 'sports', 'gardening', 'gardening']}

department_column = tf.feature_column.categorical_column_with_vocabulary_list(
        'department', ['sports', 'gardening'])
department_column = tf.feature_column.indicator_column(department_column)

columns = [
    tf.feature_column.numeric_column('sales'),
    department_column
]

inputs = tf.feature_column.input_layer(features, columns)

运行 inputs 张量会将 features 解析为一批向量。

特征列和层一样具有内部状态，因此通常需要将它们初始化。类别列会在内部使用对照表，而这些表需要单独的初始化操作 tf.tables_initializer。

In [33]:
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init,table_init))

(None, None)

In [36]:
print(department_column)
print(sess.run(inputs))

_IndicatorColumn(categorical_column=_VocabularyListCategoricalColumn(key='department', vocabulary_list=('sports', 'gardening'), dtype=tf.string, default_value=-1, num_oov_buckets=0))
[[ 1.  0.  5.]
 [ 1.  0. 10.]
 [ 0.  1.  8.]
 [ 0.  1.  9.]]


## 训练

### 定义数据

In [39]:
x = tf.constant([[1],[2],[3],[4]],dtype=tf.float32)
y_true = tf.constant([[0],[-1],[-2],[-3]],dtype=tf.float32)

### 定义模型

In [40]:
linear_model = tf.layers.Dense(units=1)
y_pred = linear_model(x)

In [41]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

print(sess.run(y_pred))

[[1.4103967]
 [2.8207934]
 [4.23119  ]
 [5.641587 ]]


### 损失

In [43]:
loss = tf.losses.mean_squared_error(labels=y_true,predictions=y_pred)
print(sess.run(loss))

32.52311


### 训练

TensorFlow 提供了优化器来执行标准的优化算法。这些优化器被实现为 tf.train.Optimizer 的子类。它们会逐渐改变每个变量，以便将损失最小化。最简单的优化算法是梯度下降法，由 tf.train.GradientDescentOptimizer 实现。它会根据损失相对于变量的导数大小来修改各个变量.

In [44]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [50]:
for epoch in range(100):
    _,loss_value = sess.run((train,loss))
    print("epoch {}:the loss is {}".format(epoch,loss_value))
    
print(sess.run(y_pred))

epoch 0:the loss is 0.022158587351441383
epoch 1:the loss is 0.02202611044049263
epoch 2:the loss is 0.021894413977861404
epoch 3:the loss is 0.02176351472735405
epoch 4:the loss is 0.021633388474583626
epoch 5:the loss is 0.021504050120711327
epoch 6:the loss is 0.02137548103928566
epoch 7:the loss is 0.021247677505016327
epoch 8:the loss is 0.021120639517903328
epoch 9:the loss is 0.02099437266588211
epoch 10:the loss is 0.02086884155869484
epoch 11:the loss is 0.02074408158659935
epoch 12:the loss is 0.02062004990875721
epoch 13:the loss is 0.020496759563684464
epoch 14:the loss is 0.020374218001961708
epoch 15:the loss is 0.02025241032242775
epoch 16:the loss is 0.020131327211856842
epoch 17:the loss is 0.020010948181152344
epoch 18:the loss is 0.01989131048321724
epoch 19:the loss is 0.01977238990366459
epoch 20:the loss is 0.0196541715413332
epoch 21:the loss is 0.019536655396223068
epoch 22:the loss is 0.019419850781559944
epoch 23:the loss is 0.019303742796182632
epoch 24:the l

In [7]:
a = tf.tanh()
with tf.Session() as sess:
    print(sess.run(a))

0.6555672
